In [1]:
import os
import sys
import numpy as np
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
from urllib import request
from urllib.request import urlretrieve
import librosa
from librosa import display
import matplotlib.pyplot as plt
import skimage
from shutil import copy2
import random
from pathlib import Path

In [2]:
track_filepath = os.path.join(sys.path[0], "metal_subgenre_data.csv")
df = pd.read_csv(track_filepath)
del df['Unnamed: 0']
del df['Unnamed: 0.1']
del df['index']
df.drop_duplicates(subset="track_id", inplace=True)

C:\Users\adamm\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (10,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df['genre'].unique()

array(['thrash metal', 'nwobhm', 'glam metal', 'mathcore', 'death metal',
       'nu metal', 'doom metal', 'hard rock', 'punk', 'industrial metal',
       'technical death metal', 'metalcore', 'hardcore', 'black metal',
       'progressive metal', 'deathcore', 'power metal',
       'melodic death metal', 'crust punk', 'stoner rock', 'goregrind',
       'grindcore', 'folk metal', 'djent', 'slam death metal',
       'sludge metal', 'funeral doom', 'screamo', 'blackgaze',
       'depressive black metal', 'symphonic metal', 'drone metal'],
      dtype=object)

In [5]:
#get authorization to access spotify

def spotify_login(cid, secret):
    client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
    return spotipy.Spotify(client_credentials_manager=client_credentials_manager)

cid = "############################" 
secret = "############################" 

sp = spotify_login(cid, secret)

In [7]:
#download track previews from urls in dataframe

mp3_folder = os.path.join(sys.path[0], "/metal_track_previews/")

preview_unavailable = []

i = 0
skip = 0
for index, row in df.iterrows():
    track_id = str(row['track_id'])
    track_url = row['preview_url'] 
    prefix = track_id[0:2] + "/"
    full_directory = mp3_folder + prefix
    
    if track_url:
        
        #check if the mp3 file already exists
        if not os.path.exists(full_directory + track_id + ".mp3"):
            
            #make the folder if we don't have it alreadt
            if not os.path.isdir(full_directory):
                os.makedirs(full_directory)
            
            #try to download the track
            try:
                urlretrieve(track_url, full_directory + track_id + ".mp3")
                i += 1
                if i % 1000 == 0:
                    print(str(i) + " tracks downloaded.")
            except Exception as e:
                print(e)
                print("Skipping track.")
                preview_unavailable.append(int(index))
              
    else:
        preview_unavailable.append(int(index))

num_downloaded = len(df.index) - len(preview_unavailable)
print("Downloaded " +  str(num_downloaded) + "/" + str(len(df.index)) + " mp3 previews.")

#df.drop(df.index[preview_unavailable], inplace=True)
#df.reset_index(drop=True, inplace=True)

Downloaded 133504/133504 mp3 previews.
